In [2]:
import boto3
import polars as pl

pl.Config.set_verbose(True)

import process_scop_sourmash_multisearch_polars

# Handwritten local modules
from polars_utils import scan_csv_sink_parquet, sink_parquet

In [3]:
s3 = boto3.resource("s3")

In [4]:
# !pip install boto3

In [5]:
csv = "s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60/sourmash/multisearch/astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.10.multisearch.csv"

In [6]:
# apq = scan_csv_sink_parquet(csv, verbose=True)
# pq

### Look at the difference in file sizes between the CSV and parquet files

In [7]:
! aws s3 ls --human-readable $csv

2024-10-11 23:54:22   87.9 GiB astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.10.multisearch.csv


In [8]:
! aws s3 cp $csv - | head

query_name,query_md5,match_name,match_md5,containment,max_containment,jaccard,intersect_hashes,prob_overlap,prob_overlap_adjusted,containment_adjusted,containment_adjusted_log10,tf_idf_score
d1dlwa_ a.1.1.1 (A:) Protozoan/bacterial hemoglobin {Ciliate (Paramecium caudatum) [TaxId: 5885]},26fce7d25101257fca79d052ec7a19c5,d3gcea_ b.33.1.0 (A:) automated matches {Nocardioides aromaticivorans [TaxId: 200618]},400e6c363be8bb5a88c54928073b54c6,0.13333333333333333,0.175,0.08187134502923976,14.0,0.00002055079627314157,4733.697725563677,0.000028166845680341016,-4.5502617856136185,0.3422001829979208
"d2gkma_ a.1.1.1 (A:) Protozoan/bacterial hemoglobin {Mycobacterium tuberculosis, HbN [TaxId: 1773]}",8e68936d4b4c7df11076541fdc12818d,d3gcea_ b.33.1.0 (A:) automated matches {Nocardioides aromaticivorans [TaxId: 200618]},400e6c363be8bb5a88c54928073b54c6,0.18018018018018017,0.25,0.11695906432748537,20.0,0.000026938530809718366,6205.056988017975,0.000029037635033507323,-4.537038757590477,0.47784874417

In [9]:
! aws s3 ls --human-readable $pq

2024-10-08 22:27:48 seanome-kmerseek
2024-10-08 22:25:14 seanome-seqera-nextflow-work


# Apply to all hp files

## Read metadata

### Read Query Metadata

In [10]:
query_metadata = pl.scan_parquet(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.query_metadata.pq"
)
query_metadata

Async thread count: 4
/home/ec2-user/miniconda3/envs/2024-kmerseek-analysis-polars/lib/python3.12/site-packages/IPython/core/formatters.py:347: UserWarning: '(default_)region' not set; polars will try to get it from bucket

Set the region manually to silence this warning.
  return method()


### Read match metadata

In [11]:
match_metadata = pl.scan_parquet(
    "s3://seanome-kmerseek/scope-benchmark/reference_files/scop.e.2.08.match_metadata.pq"
)
match_metadata

## Iterate over all files

In [12]:
# Skip ksizes 5 and 6 for now because those files are enormous, 2.3 GiB for k=5 and 175 MiB for k=6
# # -> Figure out how to use polars later
# ksizes = range(7, 21)
# moltype = "protein"


moltype_info = {
    # "protein": dict(
    #     ksizes=range(5, 21),
    #     pipeline_outdir="s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-08__protein_k5-20",
    #     analysis_outdir="s3://seanome-kmerseek/scope-benchmark/analysis-outputs/protein",
    # ),
    # "dayhoff": dict(
    #     ksizes=range(5, 21),
    #     pipeline_outdir="s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__dayhoff_k5-20",
    #     analysis_outdir="s3://seanome-kmerseek/scope-benchmark/analysis-outputs/dayhoff",
    # ),
    "hp": dict(
        ksizes=range(5, 31),
        pipeline_outdir="s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/hp",
        analysis_outdir="s3://seanome-kmerseek/scope-benchmark/analysis-outputs/hp",
    ),
}

for moltype, info in moltype_info.items():
    print(f"-- {moltype} --")
    ksizes = info["ksizes"]
    analysis_outdir = info["analysis_outdir"]
    pipeline_outdir = info["pipeline_outdir"]
    for ksize in ksizes:
        print(f"\t-- {ksize} --")
        try:
            parser = process_scop_sourmash_multisearch_polars.MultisearchParser(
                query_metadata=query_metadata,
                match_metadata=match_metadata,
                pipeline_outdir=pipeline_outdir,
                moltype=moltype,
                ksize=ksize,
                analysis_outdir=analysis_outdir,
                verbose=True,
                input_filetype="csv",
            )
            csv = parser._make_multisearch_input_file()
            print(csv)
            scan_csv_sink_parquet(csv, verbose=True)
        except FileNotFoundError:
            pass
    # break
# multisearch_metadata_filtered.head()

-- hp --
	-- 5 --
s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/hp/sourmash/multisearch/astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.5.multisearch.csv


/home/ec2-user/tmp/lkar7q3b:  23%|███████████████████▌                                                                 | 20.2G/87.8G [09:21<31:17, 36.0MB/s]


OSError: [Errno 28] No space left on device

In [ ]:
1 + 1